In [ ]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range

from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [ ]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [ ]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

In [ ]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

In [ ]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsNonLinear(df)
#     df = ATRFeature(df)
    df = KeltnerChannelsNonLinear(df)
    df = RSINonLinear(df)
    df = MACDNonLinear(df)
    df = BREAKOUT(df)
    df = ichimokuNonLinear(df)
#     df['pivots_l'] = False
#     df['pivots_h'] = False

#     df['low_perv'] = df.low.shift(1)
#     df['high_perv'] = df.high.shift(1)
#     df['low_next'] = df.low.shift(-1)
#     df['high_next'] = df.high.shift(-1)
#     df['pivots_l'] = (df.low_perv > df.low) & (df.low_next > df.low)
#     df['pivots_h'] = (df.high_perv < df.high) & (df.high_next < df.high)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

#     df.pivots_l = df.pivots_l.astype(float)
#     df.pivots_h = df.pivots_h.astype(float)
    return df

In [ ]:
df_an = featureExtraction(df_an)

In [ ]:
# df_an['SL'][i] = df_an.low[i-1] 
# df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]

# df_an['SL'][i] = df_an.high[i-1] 
# df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]

In [ ]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
# TODO: check SL and TP is resonable *********************************************************************
for i in range(1,len(df_an)): # add label for all of possible signal
    for j in range(i,len(df_an)):
        if( df_an.low[j] <= df_an.low[i-1] ):
            break
        else:
            if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                df_an.SIGNAL[i] = BUY
                break

    for j in range(i,len(df_an)):
        if( df_an.high[j] >= df_an.high[i-1] ):
            break
        else:
            if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                df_an.SIGNAL[i] = SELL
                break
                    
                    

In [ ]:
df_an.columns

In [ ]:

feature_col = [
    'BB_MA_cross', 'BB_UP_touch','BB_LW_touch',
    'EMA_cross', 'KeUp_touch', 'KeLo_touch', 
    'RSI_level',
    'MACDHIST_level',
#      'BREAK_UP', 'BREAK_DOWN',
        'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch',
        'Ten_kij_cross','Cloud_cross',
#         'full_range',# this is added ...
#        'body_lower', 'body_upper', 
#         'body_bottom_perc', 'body_top_perc',
#        'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
#     'mid_point', 'mid_point_prev_2', 'body_size_prev',
#        'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'direction',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [ ]:
X = []
Y = [] 
indeces = []
for i in range(4,len(df_an)):
    feature = []
    feature.append(df_an.pivots_h[i-1])
    feature.append(df_an.pivots_l[i-1])
    feature.append(df_an.pivots_h[i-2])
    feature.append(df_an.pivots_l[i-2])
    feature.append(df_an.pivots_h[i-3])
    feature.append(df_an.pivots_l[i-3])
    feature.append(df_an.pivots_h[i-4])
    feature.append(df_an.pivots_l[i-4])
    for fc in feature_col:
        feature.append(df_an[fc][i])
        feature.append(df_an[fc][i-1])
        feature.append(df_an[fc][i-2])
        feature.append(df_an[fc][i-3])
        feature.append(df_an[fc][i-4])

    X.append(feature)
    Y.append(df_an.SIGNAL[i])
    indeces.append(i)
        

In [ ]:
# for fc in feature_col:
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx]; idx++;")
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx-1]; idx++;")
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx-2]; idx++;")
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx-3]; idx++;")
#     print(f"feat_vec[0,idx] = feat.{fc}[bar_idx-4]; idx++;")

In [ ]:
SPLIT_INDEX = int(0.8*len(X))
X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]

In [ ]:
len(feature)

In [ ]:
clf = MLPClassifier( solver='adam', alpha=1e-3,learning_rate_init=1e-4, # adam
                    hidden_layer_sizes=(200,200), 
                    random_state=1,
                    max_iter=1000,
                    verbose=True,warm_start = True,
                    n_iter_no_change=40,
#                     early_stopping = True,
#                     validation_fraction = 0.2 # default is 0.1
                   )

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
coefs_0 = np.genfromtxt('coefs_0.csv', delimiter=',')
coefs_1 = np.genfromtxt('coefs_1.csv', delimiter=',')
coefs_2 = np.genfromtxt('coefs_2.csv', delimiter=',')
intercepts_0 = np.genfromtxt('intercepts_0.csv', delimiter=',')
intercepts_1 = np.genfromtxt('intercepts_1.csv', delimiter=',')
intercepts_2 = np.genfromtxt('intercepts_2.csv', delimiter=',')

In [ ]:
y_perdict = clf.predict(X_test)

In [ ]:
clf.score(X_train, Y_train)

In [ ]:
clf.score(X_test, Y_test)

In [ ]:
conf = confusion_matrix(Y_test, y_perdict)
conf

In [ ]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
df_test['SL'] = 0
df_test['TP'] = 0
for i,idx in enumerate(indeces_test):
    if i >1:
        if y_perdict[i] == BUY:
            df_test['SL'][idx] = df_test.low[idx-1] 
            df_test['TP'][idx] = 2*(df_test.close[idx] - df_test.low[idx-1] ) + df_test.close[idx]
            if df_test['SL'][idx] <  df_test.close[idx]:
                df_test.SIGNAL.loc[idx] = y_perdict[i]
        if  y_perdict[i] == SELL:
            df_test['SL'][idx] = df_test.high[idx-1] 
            df_test['TP'][idx] = 2*(df_test.close[idx] - df_test.high[idx-1] ) + df_test.close[idx]
            if df_test['SL'][idx] >  df_test.close[idx]:
                df_test.SIGNAL.loc[idx] = y_perdict[i]
            else: 
                print("****")
df_test.reset_index(drop=True, inplace=True)

In [ ]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio = 1 # abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])